In [21]:
import pandas as pd

In [2]:
from embedder import get_et5

/home/fernando/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
regions = pd.read_table('disprot_data/DisProt release_2022_06 with_ambiguous_evidences.tsv')

In [39]:
regions = regions[['acc','start','end','region_sequence']].drop_duplicates()

In [40]:
regions['length'] = regions.end - regions.start + 1
regions['idx'] = regions.apply(lambda row: f'{row.acc}_{row.start}_{row.end}',1)

In [43]:
regions = regions[regions.length > 20][['idx', 'region_sequence']] 

In [45]:
regions['source'] = 'disprot'

In [89]:
monomers = pd.read_table('disprot_data/monomers.tsv').rename(columns={'sequence':'region_sequence'})

monomers.region_sequence = monomers.region_sequence.str.upper()
monomers['source'] = "monomers"
monomers = monomers[~monomers.region_sequence.str.contains('X') & ~monomers.region_sequence.str.contains('B')]


In [60]:
random = pd.read_csv('disprot_data/random_sequences.tsv', sep='\t')
random = random.rename(columns={'seq':'region_sequence'})
random['source'] = "random_" + random.tipo

In [61]:
random

,idx,region_sequence,tipo,dc,source
0,unif_0,SAVCVACLYTEYTENWLKRGDMGCVDEHGSRMCYREWCDDFGGLYT...,unif,0.036667,random_unif
1,uniprot_0,RTDAGFKLFALATEAMDDITPEENDVVWFFALETSEFIVEDPIIQQ...,uniprot,0.000000,random_uniprot
2,idrs_disprot_0,TNNVPGQLQKQDKEMERFPSYSDNVTATEKSVCEEEAPVSENVMPT...,idrs,0.740000,random_idrs
3,monomers_0,NTFPSMDFKPGLCQFPLTLASLASPPKTSDLKTRVSQLGQASSQEL...,monomers,0.170000,random_monomers
4,unif_1,HPCIEAAHTYILEEWKPASEAKHLWHQEERYECYWMCECFMQFIVM...,unif,0.000000,random_unif
...,...,...,...,...,...
395,monomers_98,MFANTAAAGTVNLHVDARKFNGPWDANEEVNPMVYYVDNMEERYTD...,monomers,0.060000,random_monomers
396,unif_99,MARDYAFYFYEHVIVQENGGCGHLVDFTHLDQKSHFNLSAYNHDTT...,unif,0.000000,random_unif
397,uniprot_99,LHTSTNEALERNGLINRQYQACTPSVRCLKGDLSSILFGETMDAGI...,uniprot,0.040000,random_uniprot
398,idrs_disprot_99,YEDNYMEIEKYLALTKKRELLKSATSPLGFRKSSTARQLRVVLSGS...,idrs,0.736667,random_idrs


In [90]:
sequences = pd.concat([regions, monomers, random[['idx', 'region_sequence', 'source']]])


In [91]:
sequences.to_csv('sequences.tsv', index=False)

In [67]:
embs = get_et5(sequences.region_sequence.head(10))

Starting at 09:02:52
..........Finishing at 09:03:25
Elapsed 0:00:32.957861
3 seconds per sequence


In [81]:
import pickle
embeddings = []
CHUNK_SIZE = 500

In [94]:
for i in range(len(embeddings) * CHUNK_SIZE,len(sequences), CHUNK_SIZE):
    print(i)
    print()
    subset = sequences.region_sequence[i:i+CHUNK_SIZE]
    embs = get_et5(subset)
    embeddings.extend(embs)
    
    with open('tmp_seqs.pickle','wb') as pf:
        pickle.dump(embeddings, pf)

2500

Starting at 10:15:59
.....................................................................................................100/207
....................................................................................................200/102
....................................................................................................300/56
....................................................................................................400/22
...................................................................................................Finishing at 10:20:54
Elapsed 0:04:54.103030
0 seconds per sequence
3000

Starting at 10:20:58
.....................................................................................................100/248
....................................................................................................200/312
....................................................................................................300/184
.......................

In [101]:
sequences['embeddings'] = _emb

In [108]:
#with open('/data/fernando/elte/embeddings_dis_mono_random_25_10_df.pickle', 'bw') as pf:
#    pickle.dump(sequences, pf)

In [135]:
sequences.query('source != "random_uniprot"')

,idx,region_sequence,source,embeddings
0,P03265_294_334,EHVIEMDVTSENGQRALKEQSSKAKIVKNRWGRNVVQISNT,disprot,"[[-0.14546771, -0.22398427, -0.17456537, 0.087..."
2,P49913_134_170,LLGDFFRKSKEKIGKEFKRIVQRIKDFLRNLVPRTES,disprot,"[[0.25935963, -0.22279488, -0.22727492, 0.0090..."
5,P03045_1_107,MDAQTRRRERRAEKQAQWKAANPLLVGVSAKPVNRPILSLNRKPKS...,disprot,"[[0.09483907, -0.3427391, 0.4016453, 0.3597493..."
14,P03045_1_22,MDAQTRRRERRAEKQAQWKAAN,disprot,"[[0.07522084, -0.29852203, -0.082780525, 0.185..."
20,P03045_1_36,MDAQTRRRERRAEKQAQWKAANPLLVGVSAKPVNRP,disprot,"[[0.0069504166, -0.30938828, 0.24736862, 0.190..."
...,...,...,...,...
394,idrs_disprot_98,LVVEPRYSIIIGYRPNSCKDHDDLNRAARIAKGEAATYMNIDLPSL...,random_idrs,"[[0.029828181, -0.18922722, -0.024930023, 0.17..."
395,monomers_98,MFANTAAAGTVNLHVDARKFNGPWDANEEVNPMVYYVDNMEERYTD...,random_monomers,"[[0.1564757, 0.0012199524, 0.24375804, 0.31780..."
396,unif_99,MARDYAFYFYEHVIVQENGGCGHLVDFTHLDQKSHFNLSAYNHDTT...,random_unif,"[[0.12827691, -0.12420882, 0.25003305, 0.04886..."
398,idrs_disprot_99,YEDNYMEIEKYLALTKKRELLKSATSPLGFRKSSTARQLRVVLSGS...,random_idrs,"[[-0.06800358, -0.16696112, -0.052865274, 0.02..."


In [138]:
df = sequences.query('source != "random_uniprot"')

In [139]:
reduced_embeddings = [embedder.reduce_per_protein(pre) for pre in df.embeddings]

options = {
    'min_dist': .1,
    'spread': 8,
    'n_neighbors': 50,
    'metric': 'euclidean',
    'n_components': 2,
    'random_state': 10
}

projected_embeddings = umap_reduce(reduced_embeddings, **options)
projected_embeddings_dataframe = pd.DataFrame(
    projected_embeddings,
    columns=["component_0", "component_1"],
)
projected_embeddings_dataframe.index = list(df.idx)
projected_embeddings_dataframe["label"] = list(df.source)

/home/fernando/miniconda3/lib/python3.9/site-packages/umap/umap_.py:1356: RuntimeWarning:

divide by zero encountered in power



UMAP(n_neighbors=50, random_state=10, spread=8, verbose=1)
Tue Oct 25 12:50:04 2022 Construct fuzzy simplicial set
Tue Oct 25 12:50:04 2022 Finding Nearest Neighbors
Tue Oct 25 12:50:04 2022 Building RP forest with 9 trees
Tue Oct 25 12:50:04 2022 NN descent for 13 iterations
	 1  /  13
	 2  /  13
	 3  /  13
	Stopping threshold met -- exiting after 3 iterations
Tue Oct 25 12:50:06 2022 Finished Nearest Neighbor Search
Tue Oct 25 12:50:06 2022 Construct embedding


Epochs completed: 100%| ███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ 500/500 [00:13]

Tue Oct 25 12:50:21 2022 Finished embedding


In [147]:
figure = render_scatter_plotly(projected_embeddings_dataframe)
figure.show(renderer='iframe')

In [146]:
import plotly
import plotly.express as px

colores = ['rgb(228,26,28)',
 'rgb(55,126,184)',
  'rgb(152,78,163)',
 'rgb(77,175,74)',
 'rgb(255,127,0)',
 'rgb(255,255,51)',
 'rgb(166,86,40)',
 'rgb(247,129,191)',
 'rgb(153,153,153)']

def render_scatter_plotly(embeddings_dataframe):
    if 'label' in embeddings_dataframe.columns:
        fig = px.scatter(embeddings_dataframe,
                         x='component_0',
                         y='component_1',
                         color='label',
                         symbol='label',
                         hover_name=embeddings_dataframe.index,
                         hover_data=["label"],
                         opacity = .4,
                         color_discrete_sequence=colores
                         )
    else:
        fig = px.scatter(embeddings_dataframe,
                         x='component_0',
                         y='component_1',
                         hover_name=embeddings_dataframe.index,
                         )

    fig.update_layout(
        # Remove axes ticks and labels as they are usually not informative
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        scene=dict(
            xaxis=dict(
                showticklabels=False,
                showspikes=False,
                title=""
            ),
            yaxis=dict(
                showticklabels=False,
                showspikes=False,
                title=""
            )
        ),
    )

    return fig


In [142]:
figure = render_scatter_plotly(projected_embeddings_dataframe)
figure.show(renderer='iframe')

In [150]:
s = sequences.query('source == "monomers"')

In [151]:
s['largo'] = s.region_sequence.str.len()

/tmp/ipykernel_49241/2913622145.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [170]:
s.query('largo == 140').iloc[5,:]

idx                                                            2WY4A
region_sequence    MTKEQIQIIKDCVPILQKNGEDLTNEFYKIMFNDYPEVKPMFNMEK...
source                                                      monomers
embeddings         [[-0.07032994, -0.2915927, 0.28830814, 0.22719...
largo                                                            140
Name: 839, dtype: object

In [171]:
pd

<module 'pandas' from '/home/fernando/miniconda3/lib/python3.9/site-packages/pandas/__init__.py'>

In [172]:
projected_embeddings_dataframe

,component_0,component_1,label
P03265_294_334,28.331701,9.450675,disprot
P49913_134_170,30.083553,10.574490,disprot
P03045_1_107,22.968891,9.131040,disprot
P03045_1_22,28.895761,7.952749,disprot
P03045_1_36,26.991978,7.300606,disprot
...,...,...,...
idrs_disprot_98,14.867069,-1.470438,random_idrs
monomers_98,13.006228,21.979971,random_monomers
unif_99,12.621278,36.086349,random_unif
idrs_disprot_99,14.245030,-3.185513,random_idrs


In [174]:
df['largo'] = df.region_sequence.str.len()

/tmp/ipykernel_49241/2976090136.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [178]:
df[(df.largo > 280)& (df.largo < 320)]

,idx,region_sequence,source,embeddings,largo
104,P49918_1_316,MSDASLRSTSTMERLVARGTFPVLVRTSACRSLFGPVDHEELSREL...,disprot,"[[0.08079723, -0.1480087, -0.0036148636, 0.186...",316
376,P07214_18_302,APQQTEVAEEIVEEETVVEETGVPVGANPVQVEMGEFEDGAEETVE...,disprot,"[[-0.04136322, -0.32984945, 0.124414824, 0.159...",285
741,P10163_17_310,ESSSEDVSQEESLFLISGKPEGRRPQGGNQPQRPPPPPGKPQGPPP...,disprot,"[[-0.0020476836, -0.21037327, 0.08818779, 0.20...",294
808,P03422_1_300,MAEEQARHVKNGLECIRALKAEPIGSLAIEEAMAAWSEISDNPGQE...,disprot,"[[0.12868007, -0.20555663, -0.02795302, -0.024...",300
813,P03422_1_304,MAEEQARHVKNGLECIRALKAEPIGSLAIEEAMAAWSEISDNPGQE...,disprot,"[[0.16955136, -0.18703653, -0.024619149, -0.03...",304
...,...,...,...,...,...
394,idrs_disprot_98,LVVEPRYSIIIGYRPNSCKDHDDLNRAARIAKGEAATYMNIDLPSL...,random_idrs,"[[0.029828181, -0.18922722, -0.024930023, 0.17...",300
395,monomers_98,MFANTAAAGTVNLHVDARKFNGPWDANEEVNPMVYYVDNMEERYTD...,random_monomers,"[[0.1564757, 0.0012199524, 0.24375804, 0.31780...",300
396,unif_99,MARDYAFYFYEHVIVQENGGCGHLVDFTHLDQKSHFNLSAYNHDTT...,random_unif,"[[0.12827691, -0.12420882, 0.25003305, 0.04886...",300
398,idrs_disprot_99,YEDNYMEIEKYLALTKKRELLKSATSPLGFRKSSTARQLRVVLSGS...,random_idrs,"[[-0.06800358, -0.16696112, -0.052865274, 0.02...",300


In [180]:
reduced_embeddings = [embedder.reduce_per_protein(pre) for pre in df[(df.largo > 280)& (df.largo < 320)].embeddings]

options = {
    'min_dist': .1,
    'spread': 8,
    'n_neighbors': 50,
    'metric': 'euclidean',
    'n_components': 2,
    'random_state': 10
}

projected_embeddings = umap_reduce(reduced_embeddings, **options)
projected_embeddings_dataframe = pd.DataFrame(
    projected_embeddings,
    columns=["component_0", "component_1"],
)
projected_embeddings_dataframe.index = list(df[(df.largo > 280)& (df.largo < 320)].idx)
projected_embeddings_dataframe["label"] = list(df[(df.largo > 280)& (df.largo < 320)].source)

/home/fernando/miniconda3/lib/python3.9/site-packages/umap/umap_.py:1356: RuntimeWarning:

divide by zero encountered in power



UMAP(n_neighbors=50, random_state=10, spread=8, verbose=1)
Thu Nov  3 15:36:05 2022 Construct fuzzy simplicial set
Thu Nov  3 15:36:10 2022 Finding Nearest Neighbors
Thu Nov  3 15:36:12 2022 Finished Nearest Neighbor Search
Thu Nov  3 15:36:12 2022 Construct embedding


Epochs completed: 100%| ███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ 500/500 [00:02]

Thu Nov  3 15:36:15 2022 Finished embedding


In [181]:
render_scatter_plotly(projected_embeddings_dataframe).show(renderer='iframe')